In [1]:
import pandas as pd
import json
import numpy as np

In [4]:
base = '../../data/'
plan = pd.read_csv(base+'test_plans.csv')
plan.plans = plan.apply(lambda x: json.loads(x[2]), axis=1)

new_plan = pd.concat([plan['sid'], plan.plans.apply(pd.Series)], axis=1)
new_plan.columns = ['sid', 'a', 'b', 'c', 'd', 'e', 'f', 'g']
new_plan.set_index('sid', inplace=True)
new_plan = new_plan[['a', 'b', 'c', 'd', 'e', 'f', 'g']].stack()

new_plan = new_plan.reset_index('sid')
new_plan.reset_index(drop=True, inplace=True)
new_plan.columns = ['sid', 'plans']

new_plan['distance'] = new_plan.plans.apply(lambda x: x['distance'])
new_plan['price'] = new_plan.plans.apply(lambda x: x['price'])
new_plan['eta'] = new_plan.plans.apply(lambda x: x['eta'])
new_plan['transport_mode'] = new_plan.plans.apply(lambda x: x['transport_mode'])

new_plan.head()

,sid,plans,distance,price,eta,transport_mode
0,1112456,"{'distance': 465, 'price': '', 'eta': 418, 'tr...",465,,418,5
1,1112456,"{'distance': 465, 'price': '', 'eta': 140, 'tr...",465,,140,6
2,1112456,"{'distance': 462, 'price': '', 'eta': 178, 'tr...",462,,178,3
3,1413458,"{'distance': 28846, 'price': 600, 'eta': 4779,...",28846,600,4779,2
4,1413458,"{'distance': 18743, 'price': '', 'eta': 3098, ...",18743,,3098,3


In [5]:
new_plan[['sid', 'distance', 'price', 'eta', 'transport_mode']].to_csv('../../data/processed/test_plan_processed.csv', index=False)

In [8]:
new_plan2 = pd.read_csv(base+'processed/test_plan_processed.csv')
new_plan2.fillna(-1, inplace=True)

agg_format = pd.DataFrame()
agg_format['agg_dis'] = new_plan2.groupby('sid').apply(lambda x: x.distance.values)
agg_format['agg_price'] = new_plan2.groupby('sid').apply(lambda x: x.price.values)
agg_format['agg_eta'] = new_plan2.groupby('sid').apply(lambda x: x.eta.values)
agg_format['agg_mode'] = new_plan2.groupby('sid').apply(lambda x: x.transport_mode.values)

agg_format = agg_format.reset_index()
agg_format['mode_num'] = agg_format.agg_mode.apply(lambda x: len(x))
agg_format['first_dis'] = agg_format.agg_dis.apply(lambda x: x[0])
agg_format['first_price'] = agg_format.agg_price.apply(lambda x: x[0])
agg_format['first_eta'] = agg_format.agg_eta.apply(lambda x: x[0])
agg_format['first_mode'] = agg_format.agg_mode.apply(lambda x: x[0])
agg_format['last_mode'] = agg_format.agg_mode.apply(lambda x: x[-1])
agg_format['mean_dis'] = agg_format.agg_dis.apply(lambda x: np.mean(x))
agg_format['min_dis'] = agg_format.agg_dis.apply(lambda x: np.min(x))
agg_format['max_dis'] = agg_format.agg_dis.apply(lambda x: np.max(x))
agg_format['mean_price'] = agg_format.agg_price.apply(lambda x: np.mean(x))
agg_format['min_price'] = agg_format.agg_price.apply(lambda x: np.min(x))
agg_format['max_price'] = agg_format.agg_price.apply(lambda x: np.max(x))
agg_format['mean_eta'] = agg_format.agg_eta.apply(lambda x: np.mean(x))
agg_format['min_eta'] = agg_format.agg_eta.apply(lambda x: np.min(x))
agg_format['max_eta'] = agg_format.agg_eta.apply(lambda x: np.max(x))

agg_format.head()

,sid,agg_dis,agg_price,agg_eta,agg_mode,mode_num,first_dis,first_price,first_eta,first_mode,last_mode,mean_dis,min_dis,max_dis,mean_price,min_price,max_price,mean_eta,min_eta,max_eta
0,420123,"[22691, 21483, 21483, 21328]","[700.0, -1.0, 5500.0, 400.0]","[4888, 2303, 2543, 6046]","[7, 3, 4, 1]",4,22691,700.0,4888,7,1,21746.25,21328,22691,1649.75,-1.0,5500.0,3945.0,2303,6046
1,420125,"[20310, 20310, 21465, 21541, 20419]","[-1.0, 5900.0, 500.0, 700.0, 400.0]","[1484, 1784, 3925, 4397, 5206]","[3, 4, 9, 7, 1]",5,20310,-1.0,1484,3,1,20809.00,20310,21541,1499.80,-1.0,5900.0,3359.2,1484,5206
2,420127,"[22706, 21745, 21745, 21372]","[700.0, -1.0, 5700.0, 400.0]","[4883, 2233, 2413, 6267]","[7, 3, 4, 1]",4,22706,700.0,4883,7,1,21892.00,21372,22706,1699.75,-1.0,5700.0,3949.0,2233,6267
3,420129,"[22378, 20658, 20658, 21555, 20964]","[700.0, -1.0, 5300.0, 500.0, 400.0]","[4594, 2165, 2465, 3861, 5693]","[7, 3, 4, 9, 1]",5,22378,700.0,4594,7,1,21242.60,20658,22378,1379.80,-1.0,5300.0,3755.6,2165,5693
4,420134,"[1148, 1148, 1046, 1046]","[-1.0, 1300.0, -1.0, -1.0]","[125, 125, 315, 895]","[3, 4, 6, 5]",4,1148,-1.0,125,3,5,1097.00,1046,1148,324.25,-1.0,1300.0,365.0,125,895


In [9]:
agg_format.to_pickle('../../data/processed/test_plan_change_format.pkl')